In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sma
import unidecode as ud

In [2]:
data = pd.read_csv("./music.csv", sep = ",")
#pd.set_option('display.max_rows', data.shape[0]+1)

In [3]:
data.insert(3,'release_date',(data['released_year'].astype(str) + '-' + \
                        data['released_month'].astype(str).str.zfill(2) + '-' + \
                        data['released_day'].astype(str).str.zfill(2)))

columns2drop= ["released_year", 
               "released_month", 
               "released_day",
               "cover_url",
               "artist_count",
               "in_deezer_playlists",
               "in_apple_playlists",
               "in_spotify_charts",
               "in_apple_charts",
               "in_deezer_charts",
               "in_shazam_charts"

              ]

data = data.drop(columns=columns2drop, errors='ignore')
data

,track_name,artist(s)_name,release_date,in_spotify_playlists,streams,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2023-07-14,553,141381703,125,B,Major,80,89,83,31,0,8,4
1,LALA,Myke Towers,2023-03-23,1474,133716286,92,C#,Major,71,61,74,7,0,10,4
2,vampire,Olivia Rodrigo,2023-06-30,1397,140003974,138,F,Major,51,32,53,17,0,31,6
3,Cruel Summer,Taylor Swift,2019-08-23,7858,800840817,170,A,Major,55,58,72,11,0,11,15
4,WHERE SHE GOES,Bad Bunny,2023-05-18,3133,303236322,144,A,Minor,65,23,80,14,63,11,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
898,My Mind & Me,Selena Gomez,2022-11-03,953,91473363,144,A,Major,60,24,39,57,0,8,3
899,Bigger Than The Whole Sky,Taylor Swift,2022-10-21,1180,121871870,166,F#,Major,42,7,24,83,1,12,6
900,A Veces (feat. Feid),"Feid, Paulo Londra",2022-11-03,573,73513683,92,C#,Major,80,81,67,4,0,8,6
901,En La De Ella,"Feid, Sech, Jhayco",2022-10-20,1320,133895612,97,C#,Major,82,67,77,8,0,12,5


In [4]:
data_track_name = data['track_name']
data_artists_name = data ['artist(s)_name']
data_track_name.to_csv('track_names.csv', index=False)
data_artists_name.to_csv('artists', index = False)

In [5]:
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
 
client_id = 'b5f988fe473d4a5ea9aacfb79665fb1b'
client_secret = '11a4b27d7c2d4026882d0412744489b1'

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [6]:
#result = sp.search(q='Igualito a Mi Apï¿', type='track', limit=1)

#result

In [6]:
duration_s_array = []
for i in range(0, len(data)):
    #print('Processing item',i)
    result = sp.search(q=data["track_name"][i], type='track', limit=3)
    duration_s = result['tracks']['items'][0]['duration_ms']/1000
    
    a = data['artist(s)_name'][i]
    artists = [s.strip() for s in a.split(",")]
    #artists = [ud.unidecode(ar) for ar in artists]
    
    found_match = False
    for track in result['tracks']['items']:
        q_artists = [artist['name'] for artist in track['artists']]
        #q_artists = [ud.unidecode(ar) for ar in q_artists]
    

        if any(item in q_artists for item in artists):
            found_match = True
            duration_s_array.append(track['duration_ms']/1000)
            
            break
    #print(i, '-', found_match, '|', q_artists)
    if not found_match:
        duration_s_array.append(None)
        

        
    #print(artists_array)
    #print("\n")
    #print(i, "\t",duration_s, 'd =', data["artist(s)_name"][i], " q =", result['tracks']['items'][0]['artists'][0]['name'])
    #print(i, "\t",duration_s, '\td =', data["release_date"][i], "\tq =", result['tracks']['items'][0]['album']['release_date'], '\td =', data["artist(s)_name"][i], "\tq =", result['tracks']['items'][0]['artists'][0]['name'])
    
data.insert(4,'duration_s', duration_s_array)
data

,track_name,artist(s)_name,release_date,in_spotify_playlists,duration_s,streams,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2023-07-14,553,183.550,141381703,125,B,Major,80,89,83,31,0,8,4
1,LALA,Myke Towers,2023-03-23,1474,197.920,133716286,92,C#,Major,71,61,74,7,0,10,4
2,vampire,Olivia Rodrigo,2023-06-30,1397,219.724,140003974,138,F,Major,51,32,53,17,0,31,6
3,Cruel Summer,Taylor Swift,2019-08-23,7858,178.426,800840817,170,A,Major,55,58,72,11,0,11,15
4,WHERE SHE GOES,Bad Bunny,2023-05-18,3133,231.704,303236322,144,A,Minor,65,23,80,14,63,11,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
898,My Mind & Me,Selena Gomez,2022-11-03,953,147.597,91473363,144,A,Major,60,24,39,57,0,8,3
899,Bigger Than The Whole Sky,Taylor Swift,2022-10-21,1180,218.502,121871870,166,F#,Major,42,7,24,83,1,12,6
900,A Veces (feat. Feid),"Feid, Paulo Londra",2022-11-03,573,191.555,73513683,92,C#,Major,80,81,67,4,0,8,6
901,En La De Ella,"Feid, Sech, Jhayco",2022-10-20,1320,230.653,133895612,97,C#,Major,82,67,77,8,0,12,5


In [8]:
missing_values = data.isnull().sum()
print(missing_values)

track_name               0
artist(s)_name           0
release_date             0
in_spotify_playlists     0
duration_s              89
streams                  0
bpm                      0
key                     94
mode                     0
danceability_%           0
valence_%                0
energy_%                 0
acousticness_%           0
instrumentalness_%       0
liveness_%               0
speechiness_%            0
dtype: int64


In [9]:
key_array = np.array(data['key'].unique())
key_array = np.delete(key_array, [6])
key_array

array(['B', 'C#', 'F', 'A', 'D', 'F#', 'G#', 'G', 'E', 'A#', 'D#'],
      dtype=object)

In [10]:
key_map = {key: i for i, key in enumerate(key_array)}
data['key'] = data['key'].map(key_map).fillna(-1).astype(int).replace(-1, np.nan)
data

,track_name,artist(s)_name,release_date,in_spotify_playlists,duration_s,streams,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2023-07-14,553,183.550,141381703,125,0.0,Major,80,89,83,31,0,8,4
1,LALA,Myke Towers,2023-03-23,1474,197.920,133716286,92,1.0,Major,71,61,74,7,0,10,4
2,vampire,Olivia Rodrigo,2023-06-30,1397,219.724,140003974,138,2.0,Major,51,32,53,17,0,31,6
3,Cruel Summer,Taylor Swift,2019-08-23,7858,178.426,800840817,170,3.0,Major,55,58,72,11,0,11,15
4,WHERE SHE GOES,Bad Bunny,2023-05-18,3133,231.704,303236322,144,3.0,Minor,65,23,80,14,63,11,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
898,My Mind & Me,Selena Gomez,2022-11-03,953,147.597,91473363,144,3.0,Major,60,24,39,57,0,8,3
899,Bigger Than The Whole Sky,Taylor Swift,2022-10-21,1180,218.502,121871870,166,5.0,Major,42,7,24,83,1,12,6
900,A Veces (feat. Feid),"Feid, Paulo Londra",2022-11-03,573,191.186,73513683,92,1.0,Major,80,81,67,4,0,8,6
901,En La De Ella,"Feid, Sech, Jhayco",2022-10-20,1320,230.653,133895612,97,1.0,Major,82,67,77,8,0,12,5


In [11]:
data = data.interpolate()
data['key'] = data['key'].astype(int)


In [12]:
key_map = {i: key for i, key in enumerate(key_array)}
data['key'] = data['key'].map(key_map)
data

,track_name,artist(s)_name,release_date,in_spotify_playlists,duration_s,streams,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2023-07-14,553,183.550,141381703,125,B,Major,80,89,83,31,0,8,4
1,LALA,Myke Towers,2023-03-23,1474,197.920,133716286,92,C#,Major,71,61,74,7,0,10,4
2,vampire,Olivia Rodrigo,2023-06-30,1397,219.724,140003974,138,F,Major,51,32,53,17,0,31,6
3,Cruel Summer,Taylor Swift,2019-08-23,7858,178.426,800840817,170,A,Major,55,58,72,11,0,11,15
4,WHERE SHE GOES,Bad Bunny,2023-05-18,3133,231.704,303236322,144,A,Minor,65,23,80,14,63,11,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
898,My Mind & Me,Selena Gomez,2022-11-03,953,147.597,91473363,144,A,Major,60,24,39,57,0,8,3
899,Bigger Than The Whole Sky,Taylor Swift,2022-10-21,1180,218.502,121871870,166,F#,Major,42,7,24,83,1,12,6
900,A Veces (feat. Feid),"Feid, Paulo Londra",2022-11-03,573,191.186,73513683,92,C#,Major,80,81,67,4,0,8,6
901,En La De Ella,"Feid, Sech, Jhayco",2022-10-20,1320,230.653,133895612,97,C#,Major,82,67,77,8,0,12,5


In [13]:
missing_values = data.isnull().sum()
print(missing_values)
data.to_csv('./music_v2.csv', index=False) 
#data

track_name              0
artist(s)_name          0
release_date            0
in_spotify_playlists    0
duration_s              0
streams                 0
bpm                     0
key                     0
mode                    0
danceability_%          0
valence_%               0
energy_%                0
acousticness_%          0
instrumentalness_%      0
liveness_%              0
speechiness_%           0
dtype: int64


In [57]:
aux_list = []
for i in range(0, len(data)):
    if i == 539:
         aux_list.append(float((int(data['streams'][i-1]) + int(data['streams'][i+1]))/2 )/100000)  
    else:
        aux_list.append(float(data['streams'][i])/1000000)
#print(aux_list)
data.insert(5,'streams_mill', aux_list)


In [58]:
data

,track_name,artist(s)_name,release_date,in_spotify_playlists,duration_s,streams_mill,streams,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2023-07-14,553,183.550,141.381703,141381703,125,B,Major,80,89,83,31,0,8,4
1,LALA,Myke Towers,2023-03-23,1474,197.920,133.716286,133716286,92,C#,Major,71,61,74,7,0,10,4
2,vampire,Olivia Rodrigo,2023-06-30,1397,219.724,140.003974,140003974,138,F,Major,51,32,53,17,0,31,6
3,Cruel Summer,Taylor Swift,2019-08-23,7858,178.426,800.840817,800840817,170,A,Major,55,58,72,11,0,11,15
4,WHERE SHE GOES,Bad Bunny,2023-05-18,3133,231.704,303.236322,303236322,144,A,Minor,65,23,80,14,63,11,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
898,My Mind & Me,Selena Gomez,2022-11-03,953,147.597,91.473363,91473363,144,A,Major,60,24,39,57,0,8,3
899,Bigger Than The Whole Sky,Taylor Swift,2022-10-21,1180,218.502,121.871870,121871870,166,F#,Major,42,7,24,83,1,12,6
900,A Veces (feat. Feid),"Feid, Paulo Londra",2022-11-03,573,191.186,73.513683,73513683,92,C#,Major,80,81,67,4,0,8,6
901,En La De Ella,"Feid, Sech, Jhayco",2022-10-20,1320,230.653,133.895612,133895612,97,C#,Major,82,67,77,8,0,12,5


In [83]:
y = np.array(data[['streams_mill', 'in_spotify_playlists']])
a = np.array(data[['danceability_%', 'valence_%', 'energy_%', 'liveness_%', 'speechiness_%']])
z = sma.add_constant(a)

zt_z = z.T @ z
zt_y = z.T @ y
zt_z_inv = np.linalg.inv(zt_z)

beta_p = zt_z_inv @ zt_y


In [84]:
print(beta_p)

[[ 8.78236723e+02  8.45171357e+03]
 [-4.16224523e+00 -6.26412352e+01]
 [ 3.28558697e-01  3.03803378e+00]
 [ 1.87386515e-01  3.35309573e+01]
 [-2.36925198e+00 -3.49379343e+01]
 [-5.61649836e+00 -5.59669617e+01]]


In [85]:
model = sma.OLS(data['streams_mill'], z)
results = model.fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           streams_mill   R-squared:                       0.024
Model:                            OLS   Adj. R-squared:                  0.018
Method:                 Least Squares   F-statistic:                     4.392
Date:                Wed, 20 Nov 2024   Prob (F-statistic):           0.000588
Time:                        01:51:29   Log-Likelihood:                -7029.1
No. Observations:                 903   AIC:                         1.407e+04
Df Residuals:                     897   BIC:                         1.410e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        878.2367    110.733      7.931      0.000     660.910    1095.564
x1            -4.1622      1.480     -2.812      0.005      -7.068      -1.257
x2             0.3286      0.946      0.347      0.728      -1.528       2.185
x3             0.1874      1.266      0.148      0.882      -2.296       2.671
x4            -2.3693      1.459     -1.624      0.105      -5.233       0.494
x5            -5.6165      2.029     -2.768      0.006      -9.599      -1.634
==============================================================================
Omnibus:                      412.417   Durbin-Watson:                   1.543
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2004.119
Skew:                           2.113   Prob(JB):                         0.00
Kurtosis:                       8.950   Cond. No.                         628.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [86]:
model = sma.OLS(data['in_spotify_playlists'], z)
results = model.fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                             
================================================================================
Dep. Variable:     in_spotify_playlists   R-squared:                       0.023
Model:                              OLS   Adj. R-squared:                  0.018
Method:                   Least Squares   F-statistic:                     4.286
Date:                  Wed, 20 Nov 2024   Prob (F-statistic):           0.000738
Time:                          01:51:32   Log-Likelihood:                -9391.2
No. Observations:                   903   AIC:                         1.879e+04
Df Residuals:                       897   BIC:                         1.882e+04
Df Model:                             5                                         
Covariance Type:              nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       8451.7136   1514.741      5.580      0.000    5478.865    1.14e+04
x1           -62.6412     20.251     -3.093      0.002    -102.385     -22.897
x2             3.0380     12.940      0.235      0.814     -22.357      28.433
x3            33.5310     17.312      1.937      0.053      -0.446      67.508
x4           -34.9379     19.959     -1.750      0.080     -74.111       4.235
x5           -55.9670     27.756     -2.016      0.044    -110.440      -1.494
==============================================================================
Omnibus:                      542.904   Durbin-Watson:                   1.777
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4030.869
Skew:                           2.765   Prob(JB):                         0.00
Kurtosis:                      11.750   Cond. No.                         628.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [87]:
z = np.array(data[['danceability_%', 'valence_%', 'energy_%', 'liveness_%', 'speechiness_%', 'acousticness_%', "instrumentalness_%"]])
z = sma.add_constant(z)
model = sma.OLS(data['streams_mill'], z)
results = model.fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           streams_mill   R-squared:                       0.030
Model:                            OLS   Adj. R-squared:                  0.022
Method:                 Least Squares   F-statistic:                     3.889
Date:                Wed, 20 Nov 2024   Prob (F-statistic):           0.000348
Time:                        01:53:16   Log-Likelihood:                -7026.5
No. Observations:                 903   AIC:                         1.407e+04
Df Residuals:                     895   BIC:                         1.411e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1001.8741    148.249      6.758      0.000     710.918    1292.830
x1            -4.6001      1.521     -3.024      0.003      -7.586      -1.614
x2             0.3651      0.981      0.372      0.710      -1.561       2.291
x3            -0.7083      1.581     -0.448      0.654      -3.811       2.395
x4            -2.5274      1.458     -1.733      0.083      -5.390       0.335
x5            -5.9232      2.031     -2.916      0.004      -9.909      -1.937
x6            -0.9456      0.959     -0.986      0.324      -2.827       0.936
x7            -4.5746      2.279     -2.008      0.045      -9.047      -0.102
==============================================================================
Omnibus:                      409.699   Durbin-Watson:                   1.533
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1978.715
Skew:                           2.099   Prob(JB):                         0.00
Kurtosis:                       8.913   Cond. No.                         864.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""